In [82]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import interpolate
plt.style.use(['science', 'notebook'])
from openpyxl import load_workbook

Load spreadsheets

# Question 1

Since 

* $SMR(d, r_d, E) = TMR(d,r_d,E)PSF(r^{(m)}_{d},E)-TMR(d,0,E)$
* $PSF(r^{(m)}_{d},E)= S_p(r^{(m)}_{d},E)PSF(r^{(m)}_0,E)$
* $TMR(d,0,E)=e^{-\mu(d-d_m)}$

where $(m)$ means at the position of maximum dose, and $r_d \approx r_d^{(m)}$, we get

$$SMR(d, r_d, E) = TMR(d,r_d,E)S_p(r_{d},E)PSF(r_0,E)-e^{-\mu(d-d_m)}$$




Get data:

In [137]:
d = np.array([5, 10, 20])
rd = np.array([5, 10, 30])
rd_mesh, d_mesh = np.meshgrid(rd, d)
TMRs = np.array([[0.910,0.745,0.484],
                 [0.928,0.786,0.532],
                 [0.945,0.837,0.620]]).T
Sps = np.array([[0.983,1,1.033]]*3)
mu = 0.043
PSF0 = 1.078
data = TMRs*Sps*PSF0-np.exp(-mu*(d_mesh-1.5))

Print table:

In [138]:
pd.DataFrame(data, columns=[f'SMR(d, {rdi}, 6MV)' for rdi in rd],
                   index=[f'd={di}' for di in d])

,"SMR(d, 5, 6MV)","SMR(d, 10, 6MV)","SMR(d, 30, 6MV)"
d=5,0.104026,0.140106,0.192050
d=10,0.095607,0.153458,0.238212
d=20,0.061527,0.122140,0.239060


# Question 2